<a href="https://colab.research.google.com/github/gnsandeep/CourseProject/blob/main/TSC_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.0MB/s 
     |████████████████████████████████| 1.1MB 35.2MB/s 
     |████████████████████████████████| 2.9MB 43.5MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=087268a809be2e25799e3fae2059f6d6783c71a8f7834cb0990756aa96a6d275
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
#device = torch.device("cuda")

In [6]:
twitterdata = pd.read_csv("dataPandas.csv")
twitterdata.head()

,Tweets,Labels
0,@USER @USER @USER I don't get this .. obviousl...,SARCASM
1,@USER @USER trying to protest about . Talking ...,SARCASM
2,@USER @USER @USER He makes an insane about of ...,SARCASM
3,@USER @USER Meanwhile Trump won't even release...,SARCASM
4,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,SARCASM


In [8]:
twitterdata.shape

(5000, 2)

In [10]:
twitterdata['Labels'].value_counts(normalize = True)

NOT_SARCASM    0.5
SARCASM        0.5
Name: Labels, dtype: float64